In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

def search_artist_nhaccuatui(driver, artist_name):
    """Tìm nghệ sĩ trên NhacCuaTui và lấy link trang nghệ sĩ."""
    driver.get("https://www.nhaccuatui.com/")
    time.sleep(3)

    try:
        search_box = driver.find_element(By.CSS_SELECTOR, "input#txtSearch")
        search_box.click()
        time.sleep(1)

        search_box.send_keys(artist_name)
        time.sleep(3)

        suggestions = driver.find_elements(By.CSS_SELECTOR, ".info-search .qsItem a")
        for suggestion in suggestions:
            if "nghe-si" in suggestion.get_attribute("href"):
                return suggestion.get_attribute("href")
    except Exception as e:
        print(f"Lỗi khi tìm nghệ sĩ {artist_name}: {e}")
    return None

def get_artist_songs(driver, artist_song_url):
    """Lấy danh sách bài hát của nghệ sĩ từ tất cả các trang."""
    driver.get(artist_song_url)
    time.sleep(3)
    song_list = []
    while True:
        song_blocks = driver.find_elements(By.CSS_SELECTOR, ".box-content-music-list .info_song")
        for song_block in song_blocks:
            try:
                song_name = song_block.find_element(By.CSS_SELECTOR, "h3 a").text
                song_link = song_block.find_element(By.CSS_SELECTOR, "h3 a").get_attribute("href")
                song_list.append(("N/A", song_name, song_link))
            except Exception as e:
                print(f"Lỗi khi lấy bài hát: {e}")
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "a.number[rel='next']")
            next_link = next_button.get_attribute("href")
            if next_link:
                driver.get(next_link)
                time.sleep(3)
            else:
                break
        except:
            break
    return song_list

def get_artist_albums(driver, artist_album_url):
    """Lấy danh sách Album của nghệ sĩ."""
    driver.get(artist_album_url)
    time.sleep(3)

    album_list = []
    while True:
        album_blocks = driver.find_elements(By.CSS_SELECTOR, ".box-left-album a.box_absolute")
        for album in album_blocks:
            try:
                album_link = album.get_attribute("href")
                album_list.append(album_link)
            except Exception as e:
                print(f"Lỗi khi lấy album: {e}")
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "a.number[rel='next']")
            next_link = next_button.get_attribute("href")
            if next_link:
                driver.get(next_link)
                time.sleep(3)
            else:
                break
        except:
            break
    return album_list

def get_album_songs(driver, album_url):
    """Lấy danh sách bài hát trong Album."""
    driver.get(album_url)
    time.sleep(3)
    try:
        album_name = driver.find_element(By.CSS_SELECTOR, ".name_title").text
        song_elements = driver.find_elements(By.CSS_SELECTOR, "li[id^='itemSong_']")
        album_songs = [(album_name,
                        s.find_element(By.CSS_SELECTOR, "meta[itemprop='name']").get_attribute("content"),
                        s.find_element(By.CSS_SELECTOR, "meta[itemprop='url']").get_attribute("content"))
                        for s in song_elements]
        return album_songs
    except Exception as e:
        print(f"Lỗi khi lấy bài hát từ album {album_url}: {e}")
    return []

driver = webdriver.Chrome()
artist = "Ngắn"

artist_page = search_artist_nhaccuatui(driver, artist)
if artist_page:
    print(f"Trang nghệ sĩ: {artist_page}")

    artist_song_page = artist_page.replace(".html", ".bai-hat.html")
    artist_album_page = artist_page.replace(".html", ".playlist.html")

    artist_songs = get_artist_songs(driver, artist_song_page)
    print(pd.DataFrame(artist_songs, columns=["Album", "Tên bài hát", "Link bài hát"]))
    
    albums = get_artist_albums(driver, artist_album_page)
    print(f"Tìm thấy {len(albums)} album.")
    artist_songs = [(song_name, song_name, song_link) for _, song_name, song_link in artist_songs]
    all_songs = artist_songs[:]
    for album_link in albums:
        album_songs = get_album_songs(driver, album_link)
        all_songs.extend(album_songs)

    df_nct = pd.DataFrame(all_songs, columns=["album_name", "tracklist(danh sách bài hát)", "Link bài hát"])
    df_nct.to_excel(f"{artist}_NhacCuaTui_Songs.xlsx", index=False)
    print(f"Lấy được {len(all_songs)} bài hát của {artist}, lưu vào file Excel!")
else:
    print("Không tìm thấy nghệ sĩ.")
driver.quit()


Trang nghệ sĩ: https://www.nhaccuatui.com/nghe-si-ngan.html
